In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath('..')))

In [2]:
import pandas as pd
from scripts.database import DbConn, DatabaseConn

In [3]:
detected_image = pd.read_csv('../image/final_results/detected_images.csv')

In [4]:
detected_image.head()

,Image,xmin,ymin,xmax,ymax,confidence,class
0,@lobelia4cosmetics_11062.jpg,183.597168,0.000000,1111.034180,1264.329590,0.976964,Foodpackage
1,@lobelia4cosmetics_11061.jpg,242.018066,0.000000,1087.031128,1280.000000,0.668643,Lotion_Moisturizers
2,@lobelia4cosmetics_11060.jpg,98.514877,11.680565,701.334839,606.288147,0.974131,Lotion_Moisturizers
3,@lobelia4cosmetics_11059.jpg,167.451294,0.384521,861.117920,1278.018555,0.897209,Lotion_Moisturizers
4,@lobelia4cosmetics_11058.jpg,420.406372,115.573479,686.156494,825.498718,0.952333,Supplement


In [5]:
detected_image.shape

(3128, 7)

In [6]:
dbconn = DbConn()
db_data = dbconn.read_data('lobeliapha_transformed')

Connected to database


c:\Users\Hp\Desktop\Kifiya\week7\ETH-MedData-Warehouse\scripts\database.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f"SELECT * FROM {table_name}", self.conn)


In [7]:
db_data = pd.DataFrame(db_data, columns=['ID', 'date', 'product_name', 'weight', 'price', 'Telegram Address', 'address', 'Phone Number', 'Delivery Fee', 'Media Path'])
db_data.head()

,ID,date,product_name,weight,price,Telegram Address,address,Phone Number,Delivery Fee,Media Path
0,12401,2024-10-10 14:47:05,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12401.jpg
1,12380,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12380.jpg
2,12378,2024-10-10 04:07:50,nido usa,2.2kg,5900,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12378.jpg
3,12367,2024-10-09 15:22:01,nido,1.8kg,4500,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12367.jpg
4,12366,2024-10-09 12:44:53,sulfur8,205g,3000,https://t.me/lobelia4cosmetics,adress:- infront of bole medhanialem high school,0911562031,70 - 200 birr,photos\@lobelia4cosmetics_12366.jpg


In [36]:
# Extracting ID from the detection results table to create relation with other tables
detected_image['ID'] = detected_image['Image'].apply(lambda x: x.split('_')[1].split('.')[0])
detected_image.loc[:, 'ID'] = detected_image['ID'].astype(str)
detected_images = detected_image.drop_duplicates(subset=['ID'], keep='first')

In [37]:
detected_images.head()

,Image,xmin,ymin,xmax,ymax,confidence,class,ID
0,@lobelia4cosmetics_11062.jpg,183.597168,0.000000,1111.034180,1264.329590,0.976964,Foodpackage,11062
1,@lobelia4cosmetics_11061.jpg,242.018066,0.000000,1087.031128,1280.000000,0.668643,Lotion_Moisturizers,11061
2,@lobelia4cosmetics_11060.jpg,98.514877,11.680565,701.334839,606.288147,0.974131,Lotion_Moisturizers,11060
3,@lobelia4cosmetics_11059.jpg,167.451294,0.384521,861.117920,1278.018555,0.897209,Lotion_Moisturizers,11059
4,@lobelia4cosmetics_11058.jpg,420.406372,115.573479,686.156494,825.498718,0.952333,Supplement,11058


In [38]:
print(detected_images['ID'].dtype)  # Check data type of ID in detected_images
print(db_data['ID'].dtype)  # Check data type of ID in db_data
Ids = detected_images['ID'].values
print(type(Ids))  # Check data type of Ids

object
object
<class 'numpy.ndarray'>


In [39]:
print(set(detected_images['ID'].values))  # Print unique values of ID in detected_images
print(set(db_data['ID'].values))  # Print unique values of ID in db_data

{'10117', '10225', '11820', '12291', '10457', '10698', '10531', '11137', '12283', '10801', '10172', '11989', '12287', '10525', '10288', '11726', '11563', '11172', '10168', '10080', '11180', '10478', '11569', '11433', '10729', '11636', '11340', '10399', '10866', '12239', '12229', '12163', '10730', '11714', '11911', '10260', '10098', '10231', '12383', '10378', '11592', '11646', '11074', '11076', '12285', '12078', '11704', '11737', '10861', '10255', '12369', '11096', '10329', '10772', '11018', '11028', '11274', '10576', '10382', '11757', '11654', '11585', '10760', '11879', '10938', '10412', '11116', '10264', '10355', '10790', '11665', '10314', '10470', '10533', '10507', '11222', '12421', '11524', '12047', '12335', '10618', '11703', '11951', '11787', '10205', '11645', '12255', '11073', '10258', '11208', '11758', '11693', '12303', '10640', '11906', '11847', '10530', '10744', '11002', '10124', '11232', '10310', '11106', '10182', '11696', '10330', '12395', '10232', '10612', '10290', '11664', 

In [40]:
# Print unique IDs
print("Unique IDs in detected_images:")
print(set(detected_images['ID'].values))
print("\nUnique IDs in db_data:")
print(set(db_data['ID'].values))

Unique IDs in detected_images:
{'10117', '10225', '11820', '12291', '10457', '10698', '10531', '11137', '12283', '10801', '10172', '11989', '12287', '10525', '10288', '11726', '11563', '11172', '10168', '10080', '11180', '10478', '11569', '11433', '10729', '11636', '11340', '10399', '10866', '12239', '12229', '12163', '10730', '11714', '11911', '10260', '10098', '10231', '12383', '10378', '11592', '11646', '11074', '11076', '12285', '12078', '11704', '11737', '10861', '10255', '12369', '11096', '10329', '10772', '11018', '11028', '11274', '10576', '10382', '11757', '11654', '11585', '10760', '11879', '10938', '10412', '11116', '10264', '10355', '10790', '11665', '10314', '10470', '10533', '10507', '11222', '12421', '11524', '12047', '12335', '10618', '11703', '11951', '11787', '10205', '11645', '12255', '11073', '10258', '11208', '11758', '11693', '12303', '10640', '11906', '11847', '10530', '10744', '11002', '10124', '11232', '10310', '11106', '10182', '11696', '10330', '12395', '1023

In [41]:
Ids = detected_images['ID'].values
product_col = ['ID', 'product_name', 'weight', 'price', 'address', 'Phone Number']
Products = db_data[db_data['ID'].isin(detected_images['ID'].values)]
Products.set_index('ID', inplace=True)

In [42]:
print([id for id in detected_images['ID'].values if id in db_data['ID'].values])

['11062', '11061', '11059', '11057', '11049', '11048', '11045', '11044', '11041', '11039', '11030', '11027', '11023', '11022', '11020', '11019', '11016', '11002', '10998', '10997', '10996', '10994', '10993', '10992', '10990', '10984', '10967', '10962', '10958', '10956', '10936', '10933', '10930', '10929', '10926', '10916', '10915', '10914', '10912', '10909', '10875', '10874', '10870', '10869', '10867', '10862', '10853', '10847', '10842', '10830', '10816', '10815', '10814', '10812', '10811', '10805', '10804', '10803', '10801', '10800', '10799', '10797', '10796', '10795', '10778', '10772', '10746', '10735', '10732', '10726', '10725', '10723', '10721', '10720', '10717', '10716', '10713', '10672', '10671', '10670', '10668', '10663', '10650', '10645', '10622', '10621', '10620', '10618', '10616', '10615', '10614', '10613', '10611', '10610', '10607', '10606', '10604', '10601', '10598', '10597', '10596', '10595', '10593', '10588', '10575', '10573', '10554', '10547', '10546', '10543', '10542', 

In [43]:
# Creating a table that is only related to Telegram
Ids = detected_image['ID'].values
telegram_col = ['ID', 'date', 'Telegram Address', 'Media Path']
Telegram = db_data[telegram_col]
Telegram = Telegram[Telegram['ID'].isin(Ids)]
Telegram.set_index('ID', inplace=True)

In [44]:
# This is to select only the images that are detected and are found in the above cleaned tables from the database
Ids = Products.index
detected_images = detected_image[detected_image['ID'].isin(Ids)]
detected_images.set_index('ID', inplace=True)
detected_images.columns = ['Image', 'x_min', 'y_min', 'x_max', 'y_max', 'confidence', 'label']

In [45]:
Products.shape, detected_images.shape, Telegram.shape

((470, 9), (470, 7), (470, 3))

In [46]:
databaseConn = DatabaseConn()
databaseConn.insert_dataframe_data('Products', Products)
databaseConn.insert_dataframe_data('Telegram', Telegram)
databaseConn.insert_dataframe_data('detected_images', detected_images)

Connected to database using SQLAlchemy
